# 15.1 - Installation and Setup of VoteNet

This exercise on the application of VoteNet for building detection in 3D airborne laser scanning (ALS) point clouds is organized in several notebooks. In this first notebook, the installation and setup of the network source code as well as the exercise files and notebooks are explained.

Then, the second notebook concentrates on the ISPRS dataset with the corresponding files and classes that provides the data to the network for training and prediction.

Last, the necessary modifications on the VoteNet neural network model to the specifics of the data basis of ALS data are explained. The implementation of VoteNet is based on 3D point clouds of indoor environments, so the changes for ALS data are mainly due to the larger size of the real objects and the comparably lower point density.

**Disclaimer: This exercise and notebook are based on prototypically prepared data and conducted experiments. There are therefore many aspects that could be improved, and it cannot be ruled out that there are logical and programming errors.**

## Clone the GitHub repository

In [1]:
!git clone https://github.com/facebookresearch/votenet

Cloning into 'votenet'...
remote: Enumerating objects: 134, done.
remote: Total 134 (delta 0), reused 0 (delta 0), pack-reused 134
Receiving objects: 100% (134/134), 845.31 KiB | 5.49 MiB/s, done.
Resolving deltas: 100% (30/30), done.


Take a few minutes to explore the directory structure of votenet. 

The directory "doc" contains a file that explains how to train the VoteNet network with your own data and might be worth reading.

The two directories "scannet" and "sunrgbd" are organizing files that deal with the datasets ScanNet and Sun RGB, respectively. These files and their contained classes and functions provide an abstraction of the specifics of these datasets and thereby a common interface to the dataset for the VoteNet network. In this exercise, another directory called "ISPRS" will be created that contains similar files and classes specific for the ISPRS dataset. In the second part (notebook) of this exercise, the dataset classes and the input data are explored.

The directory "model" contains the definition of the neural network. Files in this directory are modified in the third part of the exercise to adapt the network to the ALS data.

The directory "pointnet2" contains a PyTorch PointNet++ implementation that is used by VoteNet.

The directory "utils" contains some utility functions.

## Install the ISPRS Dataset Files

The VoteNet project is organized, so that it holds for each dataset a subdirectory with files containing corresponding classes and helper functions. See e.g. the subdirectories "scannet" and "sundrgbd" in the "votenet" directory. **For the provided ISPRS dataset, copy the "ISPRS.zip" file into the directory of this notebook and execute the following command to unzip the content of the folder into the directory called "ISPRS" in the subdirectory of "votenet".** (The -d parameter allows to unzip the content of the zip file into a user provided directory instead the current directory.)

In [1]:
!unzip ISPRS.zip -d ./votenet

Archive:  ISPRS.zip
   creating: ./votenet/ISPRS/
  inflating: ./votenet/ISPRS/model_util_ISPRS.py  
  inflating: ./votenet/ISPRS/ISPRS_utils.py  
  inflating: ./votenet/ISPRS/pc_util.py  
  inflating: ./votenet/ISPRS/ISPRS_detection_dataset.py  


You should now find a directory called "ISPRS" in the directory of "votenet". **Copy the notebook 15.2 into this folder ("ISPRS"), which explores the dataset classes and the input data.**

## Install Needed Packages

The dataset classes and functions output and write data in PLY format. (PLY is a common file format in the computer graphics field to output 3D models.) Therefore, the *plyfile* Python module and the *trimesh* Python module need to be installed. This is accomplished in the following with the pip package installer. The "--user" parameter installs these packages into your home directory instead of the system directory, so that no special privileges are required on your part.

In [3]:
pip install plyfile --user

You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install trimesh --user

You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


Although it is written in the output to restart the kernel to use the updated packages, you can continue on as these packages are not yet used in this notebook. Just make sure you start the other notebooks of this exercise after installing the packages (or restart the kernel in the Kernel menu if necessary).

## Backup the Neural Network Module Files

During the exercise, some files in the models directory are modified to adapt the neural network architecture. In the following, backups of these are generated with the copy (cp) command. 

The "-n" parameter makes sure that the files are not overwritten if they already exist. This prevents that if the original files are changed later and you run these commands again, the original backups are not simply overwritten and lost. Remove this parameter if you want to make new backups.

In [9]:
!cp -n ./votenet/models/backbone_module.py "./votenet/models/backbone_module BUP.py"
!cp -n ./votenet/models/loss_helper.py "./votenet/models/loss_helper BUP.py"
!cp -n ./votenet/models/proposal_module.py "./votenet/models/proposal_module BUP.py"

## Backup Training and Evaluation Files

Also, a backup of the relevant files for training and evaluation are made.

In [10]:
!cp -n ./votenet/train.py "./votenet/train BUP.py"
!cp -n ./votenet/eval.py "./votenet/eval BUP.py"

# Point Cloud Utility Model

Since the structure of the trimesh library has changed since VoteNet was implemented, a few adjustments to the Point Cloud utility module (file 'pc_util.py') are necessary. The 'io' module was renamed to 'exchange'. 

First, backup the 'pc_util.py' file.

In [1]:
!cp -n ./votenet/utils/pc_util.py "./votenet/utils/pc_util BUP.py"

Open the 'pc_util.py' file for editing, find all occurrences of 'trimesh.io.export.export_mesh(...)' and replace them by 'trimesh.exchange.export.export_mesh(...)'. There should be 3 occurrences that need to be changed in that way. 

## Setup PointNet++

The backbone module PointNet++ uses C++ and CUDA code for performance reasons. Unfortunately, the code in GitHub is not compiling anymore in its original form and some modifications are necessary. These changes are more of a technical than of a functional (programming related) nature and they will not be discussed in detail here. We have prepared the modified files and provide them in the zip file "pointnet2 fix.zip". 

First, we rename the old "pointnet2" folder.

In [1]:
!mv votenet/pointnet2 "votenet/pointnet2 BUP"

Upload the zip file ("pointnet2 fix.zip") into the folder of this notebook and unzip it into the "votenet" folder. Now you should have both a subfolder "pointnet2" and "pointnet2 BUP".

In [2]:
!unzip "pointnet2 fix.zip" -d ./votenet

Archive:  pointnet2 fix.zip
   creating: ./votenet/pointnet2/
  inflating: ./votenet/pointnet2/pointnet2_modules.py  
  inflating: ./votenet/pointnet2/pointnet2_test.py  
  inflating: ./votenet/pointnet2/pointnet2_utils.py  
  inflating: ./votenet/pointnet2/pytorch_utils.py  
  inflating: ./votenet/pointnet2/setup.py  
   creating: ./votenet/pointnet2/_ext_src/
   creating: ./votenet/pointnet2/_ext_src/include/
  inflating: ./votenet/pointnet2/_ext_src/include/ball_query.h  
  inflating: ./votenet/pointnet2/_ext_src/include/cuda_utils.h  
  inflating: ./votenet/pointnet2/_ext_src/include/group_points.h  
  inflating: ./votenet/pointnet2/_ext_src/include/interpolate.h  
  inflating: ./votenet/pointnet2/_ext_src/include/sampling.h  
  inflating: ./votenet/pointnet2/_ext_src/include/utils.h  
   creating: ./votenet/pointnet2/_ext_src/src/
  inflating: ./votenet/pointnet2/_ext_src/src/ball_query.cpp  
  inflating: ./votenet/pointnet2/_ext_src/src/ball_query_gpu.cu  
  inflating: ./votenet/

The libraries provided by PointNet++ must be compiled before they can be used. The compilations cannot be done from the Jupyter notebook, because the setup script must be called from the current directory and we cannot change the current directory within a Jupyter notebook.

Therefore, open a terminal window (in the same way as if you would create a new Jupyter notebook with the + symbol below the menu bar and choosing the Terminal option at the bottom). Within the terminal window, change to the pointnet2 folder with the cd command. (Make sure to first change the current directory to the directory this notebook is contained.)

In [ ]:
# This is not executable within this notebook
cd votenet/pointnet2

Once you changed the current directory to the one of PointNet++, call the Python script "setup.py" to install the PointNet++ package. You should see some output that also contains a number of warnings that you can ignore. 

(The --user arguments tell Python to install it in your user environment rather than in the global environment. Because the global environment is cleaned every time you log out and the instance is restarted, you would otherwise need to run the setup script every time you log into JupyterHub.)

In [ ]:
# This is not executable within this notebook
python setup.py install --user

To check if the network is now ready to run, first change the current directory to the votenet folder and execute the *main()* function of VoteNet within "votenet.py". If no data source is provided, it will run the network with random values just to check that a forward pass is possible. 

In [ ]:
# This is not executable within this notebook
cd ..
python models/votenet.py

You should see a long list of (random) tensor outputs. (The critical error/warning of GDK can be ignored as the server does not provide GDK support and also no graphical user outputs.)

If no errors occurred, the network should be generally ready to run.

**Copy notebook 15.2 into the votenet/ISPRS folder and continue now with this notebook.**

**Copy the notebooks 15.3 and 15.4 into the current folder together with notebook 15.1 for later use.**

